Combining all the data tables together for analysis and predictive modelling

In [3]:
#import required packages
import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import earthpy as et
import pandas as pd
import pickle

#read in the city info table
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT'))
CITY_COUNTRY_lat_lon = pd.read_excel('CITY_COUNTRY_lat_lon.xlsx', index_col=None)

In [4]:
#remove cities with area less than 10km2
CITY_COUNTRY_lat_lon = CITY_COUNTRY_lat_lon[CITY_COUNTRY_lat_lon['Area_2002']>9].reset_index(drop = True)

In [7]:
# aqua only 
#create empty dataframe to append info into 
all_monthly_data_df = pd.DataFrame(data = None)

for i in range(len(CITY_COUNTRY_lat_lon)):
    CITY_COUNTRY = CITY_COUNTRY_lat_lon.CITY_COUNTRY[i]
    city_name = CITY_COUNTRY_lat_lon.City[i]
    #koppen_2_letter = CITY_COUNTRY_lat_lon.Koppen_Geiger_2_Letter[i]

    #read in the SUHI parameters  
    os.chdir(os.path.join('D:\\','MODIS_8_day_LST', CITY_COUNTRY, 'MYD11A2'))    
    pickle_name = 'Parameters_Day_{}_{}.pkl'.format(CITY_COUNTRY, 'MYD11A2')
    with open(pickle_name, 'rb') as f:
        aqua_parameters_day = pickle.load(f)
    pickle_name = 'Parameters_Night_{}_{}.pkl'.format(CITY_COUNTRY, 'MYD11A2')
    with open(pickle_name, 'rb') as f:
        aqua_parameters_night = pickle.load(f)

    #read in the climate parameters
    os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT', 'ERA5', CITY_COUNTRY))
    pickle_name = '{}_clim_rur_urb'.format(city_name)
    with open(pickle_name, 'rb') as f:
        clim_df = pickle.load(f)
    
    clim_df2 = clim_df.reset_index()[['year', 'month', 'urban_evap_fract', 'urban_rh', 'rural_evap_fract', 'rural_rh',
                                     'rural_tp','urban_tp','rural_t2m','urban_t2m','rural_ssr','urban_ssr']]
    
    # get the annual average ef
    annual_ef_df = clim_df2.groupby(['year'], as_index=False).mean()[['year','urban_evap_fract','rural_evap_fract']].rename(
        columns={"urban_evap_fract": "annual_urban_ef", "rural_evap_fract": "annual_rural_ef"})

    
    #################################### AQUA DAY #############################    
    #add in the grouping the the parameters dataset to get a0 and mean SUHI
    #aqua_parameters_day
    aqua_parameters_day['year'] = pd.to_numeric(aqua_parameters_day['Year'], downcast='integer')
    aqua_parameters_day['month'] = pd.to_numeric(aqua_parameters_day['Month'], downcast='integer')
    
    group = (aqua_parameters_day.groupby(['year', 'month']).mean().reset_index()
                          [['year','month','day_a0','method_2_SUHI','quantile_a0','footprint_area']])#'buffer_5km_SUHI',
    
    aqua_mon_averages = pd.merge(group, clim_df2, on =['year', 'month'], how = 'inner').drop_duplicates()
    aqua_mon_averages = pd.merge(aqua_mon_averages, annual_ef_df, on =['year'], how = 'left')
    
    aqua_mon_averages['Overpass'] = '13:30'    
    aqua_mon_averages['City'] = city_name
    #aqua_mon_averages['Koppen_Geiger_2_Letter'] = koppen_2_letter
    
    #################################### AQUA NIGHT ##########################   
    #add in the grouping the the parameters dataset to get a0 and mean SUHI
    #aqua_parameters_night
    aqua_parameters_night['year'] = pd.to_numeric(aqua_parameters_night['Year'], downcast='integer')
    aqua_parameters_night['month'] = pd.to_numeric(aqua_parameters_night['Month'], downcast='integer')
    
    group = (aqua_parameters_night.groupby(['year', 'month']).mean().reset_index()
                          [['year','month','night_a0','method_2_SUHI','quantile_a0','footprint_area']])#,'buffer_5km_SUHI'
    
    aqua_night_mon_averages = pd.merge(group, clim_df2, on =['year', 'month'], how = 'inner').drop_duplicates()
    aqua_night_mon_averages = pd.merge(aqua_night_mon_averages, annual_ef_df, on =['year'], how = 'left')
    
    aqua_night_mon_averages['Overpass'] = '01:30'    
    aqua_night_mon_averages['City'] = city_name
    #aqua_night_mon_averages['Koppen_Geiger_2_Letter'] = koppen_2_letter    

    #renames a0s to a0 so can be joined into same df
    aqua_mon_averages = aqua_mon_averages.rename(columns={"day_a0": "a0"})   
    aqua_night_mon_averages = aqua_night_mon_averages.rename(columns={"night_a0": "a0"}) 

    all_monthly_data_df = all_monthly_data_df.append(aqua_mon_averages)
    all_monthly_data_df = all_monthly_data_df.append(aqua_night_mon_averages)

##save the table as pickle
#os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','UHI_Project_Pickle_Files','All_cities'))   
#with open('aqua_no_evi_monthly_data_df.pkl', 'wb') as f:
#    pickle.dump(all_monthly_data_df, f)

In [6]:
CITY_COUNTRY

'PIRACICABA_BRAZIL'

In [8]:
# add in the city area and eccentricity
# create a df for the areas to merge
# for years greater than 2015, use 2015 area

year_list = ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013',
                  '2014','2015']
year_nans = ['2016','2017','2018','2019','2020']
area_list = []
city_list = []
year_loop_list = []
eccentricity_list = []

for country_i, City in enumerate(CITY_COUNTRY_lat_lon.City):   
    for n, year in enumerate(year_list):
        area_list.append(CITY_COUNTRY_lat_lon['Area_{}'.format(year)][country_i])
        eccentricity_list.append(CITY_COUNTRY_lat_lon['eccen_{}'.format(year)][country_i])
        city_list.append(City)
        year_loop_list.append(year)
    for m, nan_year in enumerate(year_nans):
        area_list.append(CITY_COUNTRY_lat_lon['Area_2015'][country_i])
        eccentricity_list.append(CITY_COUNTRY_lat_lon['eccen_2015'.format(year)][country_i])
        city_list.append(City)
        year_loop_list.append(nan_year)
        
area_df = pd.DataFrame()
area_df['City'] = city_list
year_loop_list = list(map(int, year_loop_list))
area_df['year'] = year_loop_list
area_df['Area'] = area_list        
area_df['Eccentricity'] = eccentricity_list

# merge on
all_monthly_data_df = pd.merge(all_monthly_data_df, area_df, on =['City','year'], how = 'left')

##save the table as pickle
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','UHI_Project_Pickle_Files','All_cities'))   
with open('aqua_no_evi_monthly_data_df.pkl', 'wb') as f:
    pickle.dump(all_monthly_data_df, f)

In [9]:
#remove 2021
all_monthly_data_df = all_monthly_data_df[all_monthly_data_df.year != 2021]

In [10]:
# add in the evi data
# create a dataframe with all the EVI data
all_evi_df = pd.DataFrame(data = None)

for i in range(len(CITY_COUNTRY_lat_lon)):
    CITY_COUNTRY = CITY_COUNTRY_lat_lon['CITY_COUNTRY'][i] 
    city_name = CITY_COUNTRY_lat_lon['City'][i]  
    #open the city EVI df
    os.chdir(os.path.join('D:\\','MODIS_NDVI', CITY_COUNTRY))
    pickle_name = 'vi_means_df_{}.pkl'.format(CITY_COUNTRY)
    with open(pickle_name, 'rb') as f:
        vi_means_df = pickle.load(f)
    #add the city name
    vi_means_df['City'] = city_name
    #append to overall df
    all_evi_df = all_evi_df.append(vi_means_df)

#change year/ month dtypes to int so can be matched on merge
all_evi_df['Year'] = all_evi_df['Year'].apply(int)
all_evi_df['Month'] = all_evi_df['Month'].apply(int)
all_monthly_data_df['Year'] = all_monthly_data_df['year'].apply(int)
all_monthly_data_df['Month'] = all_monthly_data_df['month'].apply(int)
    
#merge the datasets
all_monthly_data_df2 = pd.merge(all_monthly_data_df, all_evi_df, how='left', left_on=['City','Year','Month'], 
                                right_on = ['City','Year','Month'])    
      
#drop useless columns 
all_monthly_data_df2 = all_monthly_data_df2.drop(columns=['Year', 'Month','Aqua_Filename', 'Terra_Filename',
                                                         'Terra_Filename_top','Aqua_Filename_top'])

#save the new dataset
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','UHI_Project_Pickle_Files','All_cities')) 
with open('aqua_all_monthly_data_df2.pkl', 'wb') as f:
    pickle.dump(all_monthly_data_df2, f)

In [7]:
#add in later? 'buffer_5km_SUHI','quantile_a0','footprint_area'

#create empty dataframe to append info into 
all_monthly_data_df = pd.DataFrame(data = None)

for i in range(len(CITY_COUNTRY_lat_lon)):
    CITY_COUNTRY = CITY_COUNTRY_lat_lon.CITY_COUNTRY[i]
    city_name = CITY_COUNTRY_lat_lon.City[i]
    koppen_2_letter = CITY_COUNTRY_lat_lon.Koppen_Geiger_2_Letter[i]

    #read in the SUHI parameters
    os.chdir(os.path.join('D:\\','MODIS_8_day_LST', CITY_COUNTRY, 'MOD11A2'))
    pickle_name = 'Parameters_Day_{}_{}.pkl'.format(CITY_COUNTRY, 'MOD11A2')
    with open(pickle_name, 'rb') as f:
        terra_parameters_day = pickle.load(f)
    pickle_name = 'Parameters_Night_{}_{}.pkl'.format(CITY_COUNTRY, 'MOD11A2')
    with open(pickle_name, 'rb') as f:
        terra_parameters_night = pickle.load(f)   
    os.chdir(os.path.join('D:\\','MODIS_8_day_LST', CITY_COUNTRY, 'MYD11A2'))    
    pickle_name = 'Parameters_Day_{}_{}.pkl'.format(CITY_COUNTRY, 'MYD11A2')
    with open(pickle_name, 'rb') as f:
        aqua_parameters_day = pickle.load(f)
    pickle_name = 'Parameters_Night_{}_{}.pkl'.format(CITY_COUNTRY, 'MYD11A2')
    with open(pickle_name, 'rb') as f:
        aqua_parameters_night = pickle.load(f)

    #read in the climate parameters
    os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT', 'ERA5', CITY_COUNTRY))
    pickle_name = '{}_clim_rur_urb'.format(city_name)
    with open(pickle_name, 'rb') as f:
        clim_df = pickle.load(f)
    
    clim_df2 = clim_df.reset_index()[['year', 'month', 'overall_evap_fract', 'overall_rh', 
                                      'urban_evap_fract', 'urban_rh', 'rural_evap_fract', 'rural_rh',
                                     'overall_tp','rural_tp','urban_tp','overall_t2m','rural_t2m','urban_t2m',
                                     'overall_ssr','rural_ssr','urban_ssr']]
    
    #################################### TERRA DAY ##############################    
    #add in the grouping the the parameters dataset to get a0 and mean SUHI
    #convert the year and month columns to integer so they are equal for the merge
    terra_parameters_day['year'] = pd.to_numeric(terra_parameters_day['Year'], downcast='integer')
    terra_parameters_day['month'] = pd.to_numeric(terra_parameters_day['Month'], downcast='integer')
    
    group = (terra_parameters_day.groupby(['year', 'month']).mean().reset_index()
                          [['year','month','day_a0','method_2_SUHI','quantile_a0']])
    
    terra_mon_averages = pd.merge(group, clim_df2, on =['year', 'month'], how = 'inner').drop_duplicates()
    
    terra_mon_averages['Overpass'] = '10:30'    
    terra_mon_averages['City'] = city_name
    terra_mon_averages['Koppen_Geiger_2_Letter'] = koppen_2_letter
    
    #################################### AQUA DAY #############################    
    #add in the grouping the the parameters dataset to get a0 and mean SUHI
    #aqua_parameters_day
    aqua_parameters_day['year'] = pd.to_numeric(aqua_parameters_day['Year'], downcast='integer')
    aqua_parameters_day['month'] = pd.to_numeric(aqua_parameters_day['Month'], downcast='integer')
    
    group = (aqua_parameters_day.groupby(['year', 'month']).mean().reset_index()
                          [['year','month','day_a0','method_2_SUHI','quantile_a0']])
    
    aqua_mon_averages = pd.merge(group, clim_df2, on =['year', 'month'], how = 'inner').drop_duplicates()
    
    aqua_mon_averages['Overpass'] = '13:30'    
    aqua_mon_averages['City'] = city_name
    aqua_mon_averages['Koppen_Geiger_2_Letter'] = koppen_2_letter
    
    ################################### TERRA NIGHT ###########################    
    #add in the grouping the the parameters dataset to get a0 and mean SUHI
    #terra_parameters_night
    terra_parameters_night['year'] = pd.to_numeric(terra_parameters_night['Year'], downcast='integer')
    terra_parameters_night['month'] = pd.to_numeric(terra_parameters_night['Month'], downcast='integer')
    
    group = (terra_parameters_night.groupby(['year', 'month']).mean().reset_index()
                          [['year','month','night_a0','method_2_SUHI','quantile_a0']])
    
    terra_night_mon_averages = pd.merge(group, clim_df2, on =['year', 'month'], how = 'inner').drop_duplicates()
    
    terra_night_mon_averages['Overpass'] = '22:30'    
    terra_night_mon_averages['City'] = city_name
    terra_night_mon_averages['Koppen_Geiger_2_Letter'] = koppen_2_letter
    
    #################################### AQUA NIGHT ##########################   
    #add in the grouping the the parameters dataset to get a0 and mean SUHI
    #aqua_parameters_night
    aqua_parameters_night['year'] = pd.to_numeric(aqua_parameters_night['Year'], downcast='integer')
    aqua_parameters_night['month'] = pd.to_numeric(aqua_parameters_night['Month'], downcast='integer')
    
    group = (aqua_parameters_night.groupby(['year', 'month']).mean().reset_index()
                          [['year','month','night_a0','method_2_SUHI','quantile_a0']])
    
    aqua_night_mon_averages = pd.merge(group, clim_df2, on =['year', 'month'], how = 'inner').drop_duplicates()
    
    aqua_night_mon_averages['Overpass'] = '01:30'    
    aqua_night_mon_averages['City'] = city_name
    aqua_night_mon_averages['Koppen_Geiger_2_Letter'] = koppen_2_letter    

    #renames a0s to a0 so can be joined into same df
    terra_mon_averages = terra_mon_averages.rename(columns={"day_a0": "a0"})
    aqua_mon_averages = aqua_mon_averages.rename(columns={"day_a0": "a0"})
    terra_night_mon_averages = terra_night_mon_averages.rename(columns={"night_a0": "a0"})
    aqua_night_mon_averages = aqua_night_mon_averages.rename(columns={"night_a0": "a0"}) 

    all_monthly_data_df = all_monthly_data_df.append(terra_mon_averages)
    all_monthly_data_df = all_monthly_data_df.append(aqua_mon_averages)
    all_monthly_data_df = all_monthly_data_df.append(terra_night_mon_averages)
    all_monthly_data_df = all_monthly_data_df.append(aqua_night_mon_averages)

#save the table as pickle
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','UHI_Project_Pickle_Files','All_cities'))   
with open('all_monthly_data_df.pkl', 'wb') as f:
    pickle.dump(all_monthly_data_df, f)

In [10]:
#add in the evi data
#create a dataframe with all the EVI data
all_evi_df = pd.DataFrame(data = None)

for i in range(len(CITY_COUNTRY_lat_lon)):
    CITY_COUNTRY = CITY_COUNTRY_lat_lon['CITY_COUNTRY'][i] 
    city_name = CITY_COUNTRY_lat_lon['City'][i]  
    #open the city EVI df
    os.chdir(os.path.join('D:\\','MODIS_NDVI', CITY_COUNTRY))
    pickle_name = 'vi_means_df_{}.pkl'.format(CITY_COUNTRY)
    with open(pickle_name, 'rb') as f:
        vi_means_df = pickle.load(f)
    #add the city name
    vi_means_df['City'] = city_name
    #append to overall df
    all_evi_df = all_evi_df.append(vi_means_df)

#change year/ month dtypes to int so can be matched on merge
all_evi_df['Year'] = all_evi_df['Year'].apply(int)
all_evi_df['Month'] = all_evi_df['Month'].apply(int)
all_monthly_data_df['Year'] = all_monthly_data_df['year'].apply(int)
all_monthly_data_df['Month'] = all_monthly_data_df['month'].apply(int)
    
#merge the datasets
all_monthly_data_df2 = pd.merge(all_monthly_data_df, all_evi_df, how='left', left_on=['City','Year','Month'], 
                                right_on = ['City','Year','Month'])    
      
#drop useless columns 
all_monthly_data_df2 = all_monthly_data_df2.drop(columns=['Year', 'Month','Aqua_Filename', 'Terra_Filename',
                                                         'Terra_Filename_top','Aqua_Filename_top'])

#save the new dataset
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','UHI_Project_Pickle_Files','All_cities')) 
with open('all_monthly_data_df2.pkl', 'wb') as f:
    pickle.dump(all_monthly_data_df2, f)

In [12]:
all_monthly_data_df2

,year,month,a0,method_2_SUHI,quantile_a0,overall_evap_fract,overall_rh,urban_evap_fract,urban_rh,rural_evap_fract,...,urban_t2m,overall_ssr,rural_ssr,urban_ssr,Overpass,City,Koppen_Geiger_2_Letter,rur_mean_evi,urb_mean_evi,pixel_reliability_percent
0,2002,1,2.906066,0.760998,3.650000,0.757847,95.929985,0.750305,95.864914,0.758065,...,291.251892,183.240585,183.293046,181.423597,10:30,Huambo,Cw,NaN,NaN,NaN
1,2002,2,2.596784,0.801647,2.140000,0.758275,95.736839,0.752681,95.705627,0.758437,...,291.766907,191.752204,191.823445,189.284798,10:30,Huambo,Cw,NaN,NaN,NaN
2,2002,3,NaN,1.568914,2.869500,0.753038,96.402504,0.745333,96.382164,0.753261,...,291.673859,183.512472,183.582602,181.083538,10:30,Huambo,Cw,NaN,NaN,NaN
3,2002,4,1.820394,0.734614,1.440000,0.793239,93.959358,0.781038,93.962883,0.793591,...,291.809235,198.372499,198.434762,196.216036,10:30,Huambo,Cw,NaN,NaN,NaN
4,2002,5,1.585061,0.229486,0.220000,0.719868,84.684525,0.731550,84.930412,0.719531,...,291.129089,214.940667,215.010496,212.522158,10:30,Huambo,Cw,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15661,2020,9,2.259935,1.288898,1.960000,0.113581,81.298790,0.101158,81.256645,0.114431,...,292.991486,211.714841,211.779518,210.769278,01:30,Bulawayo,BS,0.142561,0.143414,97.407746
15662,2020,10,1.765050,0.766030,1.694000,0.233809,82.042282,0.232444,82.009880,0.233902,...,295.421143,249.850531,249.928315,248.713337,01:30,Bulawayo,BS,0.173692,0.176158,90.179933
15663,2020,11,2.104820,0.983248,1.989333,0.346746,84.690918,0.351147,84.664955,0.346445,...,296.661865,216.163103,216.242484,215.002569,01:30,Bulawayo,BS,0.255260,0.236684,55.504727
15664,2020,12,0.968285,0.238787,1.058500,0.773027,95.608086,0.787646,95.623184,0.772028,...,293.581665,172.261565,172.344445,171.049877,01:30,Bulawayo,BS,0.376326,0.332478,52.790485
